In [1]:
import numpy as np
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import warnings
import seaborn as sns
warnings.filterwarnings('ignore')
matplotlib.rcParams['pdf.fonttype'] = 42
from datetime import datetime
import scipy.special
import xarray as xr
import glob as gb
from matplotlib.lines import Line2D
import dask

In [2]:
def read_model_data(run,voyage): 
    
    fdir = '/g/data/jk72/slf563/ACCESS/output/campaign_data/'
    data = xr.open_dataset(fdir+'{}_daily_mean_{}_vars.nc'.format(run,voyage))
    if 'z1_hybrid_height' in list(data.coords): 
        data = data.isel(z1_hybrid_height=0)
    return(data)

In [58]:
# From Matt & similar to that used in glomap mode ('easy way')

def calc_kohler_easy_way(ss):
       rh = 1.0+(ss/100.)
       temp = 298.64
       # Calculate A factor (p787, S&P1998)
       A = 0.66/temp # in microns
       lnrh = np.log(rh)
       # Calculate B factor (p788, S&P1998) [at critical droplet diameter]
       B = (4.0*A**3.)/(27.0*lnrh**2.) # in microns^
       # solute mass (g particle-1) [at critical droplet diameter]
       ms = B*98.0/(3.0*3.44e13) # in g (3.44e13 all soluble mass per particle # of dissociating ions in mols/m) 
       # convert to kg particle-1
       ms = ms/1000.0 # in kg
       # calculate particle dry volume (assume particle density of 1800 kg m-3)
       vol = ms/1800. # in m3
       act_r = ((3.0*vol)/(4.0*np.pi))**(1.0/3.0) # in m
       print('Activation diameter',2*act_r*1e9,'nm')
       return(2*act_r*1e9)


In [73]:
# Equation 10 in https://doi.org/10.5194/acp-7-1961-2007

def calc_act_d_P_and_K_way(data,ss,all_h2so4=True): 
    # Constants
    sigma = 0.0728  # Surface tension of water (N/m)
    Mw = 0.018  # Molar mass of water (kg/mol)
    R = 8.314  # Universal gas constant (J/(mol·K))
    T = 298.15  # Temperature in Kelvin (15°C)
    Pw = 997  # Density of water (kg/m³)
    
    A = (4*sigma*Mw)/(R*T*Pw) # in meters 
    Sc = 1+(ss/100) 
    
    lnSc =  np.log(Sc)
    
    B = (4*(A**3))/(27*(lnSc**2)) # in meters 

    # Caluclate the volumes... Ei
    # Have just added mixing volumes (mol/cm3) of all modes/aero types, not sure if this is right?
    VH2SO4 = data['field34102']+data['field34104']+data['field34108']+data['field34114']
    VBC = data['field34105']+data['field34109']+data['field34115']
    VOM = data['field34106']+data['field34110']+data['field34116']
    VSS = data['field34111']+data['field34117']
    # Calc sum(Ei)
    VTOT = VH2SO4 + VBC + VOM + VSS
    
    # Calculate kappa according to Petters and Kreidenweis 2007: Sum(Ei x ki)
    if all_h2so4 == False: # assume all BC and OM have a H2SO4 coating & that SS is externally mixed 
        k = VH2SO4/VTOT*0.9 + VOM/VTOT*0.9 + VBC/VTOT*0.9 + VSS/VTOT*1.28
        k = k.mean().values
        print('Assume SS is externally mixed')
    else: # If it is all internally mixed - everything is covered in H2SO4
        k = 0.9
        print('Assume SS is internally mixed')
        
    D = (B/k)**(1/3)
    print('Activation diameter',(D*1e9),'nm')
    return(D)



# Calc activation diameter

In [74]:
dataKCG = read_model_data('dg657','Kennaook-Cape Grim')
dataSyowa = read_model_data('dg657','Syowa')
ss=0.5

In [75]:
D = calc_act_d_P_and_K_way(dataKCG,ss,all_h2so4=False)
D = calc_act_d_P_and_K_way(dataSyowa,ss,all_h2so4=False)

Assume SS is externally mixed
Activation diameter 35.91545009338469 nm
Assume SS is externally mixed
Activation diameter 35.91681165475812 nm


In [76]:
D = calc_act_d_P_and_K_way(dataKCG,ss,all_h2so4=True)
D = calc_act_d_P_and_K_way(dataSyowa,ss,all_h2so4=True)

Assume SS is internally mixed
Activation diameter 39.81842771119052 nm
Assume SS is internally mixed
Activation diameter 39.81842771119052 nm


In [77]:
act_r = calc_kohler_easy_way(ss)

Activation diameter 40.16016012999456 nm
